# Classificação dos recebedores

Iniciamos com as importações padrões.

In [1]:
import numpy as np
import pandas as pd

A seguir prosseguimos com a leitura dos seguintes conjuntos de dados:
* `events` é o conjunto de todos os eventos da base de dados com eliminação das linhas duplicadas;
* `recipients` é o conjunto de todos os recebedores. Incializamos os campos `send day` com 0 e `send time` com o `string` vazio. Estes campos serão preenchidos com o dia da semana e a hora de envio para otimizar a taxa de abertura para o recebedor.
* `clusters` é o conjunto de 13 clusters encontrados por meio do algoritmo de análise de clusters.

In [2]:
events = pd.read_csv('events.csv')\
    .drop_duplicates(['id', 'timestamp', 'email_id', 'action'])
recipients = pd.read_csv('recipients.csv').set_index('id')
recipients['send day'] = 0
recipients['send time'] = ''
clusters = pd.read_csv('clusters.csv')

Para cada recebedor:
1. Se houver `action` igual a `unsubscribe` ou `spamreport` atribuir o valor -1 para `send day`, indicando para não enviar mais e-mails para o recebedor;
2. Senão verificar qual é o cluster quando ocorre a maior parte dos `open`s do recebedor;

  1. Atribuir o valor do dia da semana e a hora de envio encontrado para os campos `send day` e `send time`. Em caso de empate entre os clusters, escolher o cluster de rank mais alto. O valor de `send day` é um inteiro entre 1 e 7, sendo que 1 corresponde ao domingo e 7 ao sábado.

  2. Se não houver `open` em nenhum dos clusters deixar `send day` com o valor 0, indicando que ainda não foi possível encontrar a melhor hora para o envio para o recebedor e que há necessidade de levantar mais dados para determiná-la.

Por ser um processo demorado, a cada 1% processado o programa mostra uma mensagem na tela.

In [3]:
percentis = np.linspace(0, len(recipients), 101);
percent = 0;
curr = 0;
for id, recipient in recipients.iterrows():
    if curr > percentis[percent + 1]:
        percent += 1
        print('{}% complete'.format(percent))
    curr += 1
    unsubscriptions = events.query(
        'id == @id and (action == "unsubscribe" or action == "spamreport")')
    if unsubscriptions.size:
        recipients.loc[id, 'send day'] = -1
    else:
        clusters['count'] = 0
        opens = events.query('id == @id and action == "open"')
        classified = False
        for index2, event in opens.iterrows():
            for index1, cluster in clusters.iterrows():
                if cluster['start'] <= event['daytime'] <= cluster['end']:
                    clusters.loc[index1, 'count'] += 1
                    classified = True
        if classified:
            index_max = clusters['count'].idxmax()
            recipients.loc[id, 'send day'] = clusters.loc[index_max, 'send day']
            recipients.loc[id, 'send time'] = clusters.loc[index_max, 'send time']


1% complete
2% complete
3% complete
4% complete
5% complete
6% complete
7% complete
8% complete
9% complete
10% complete
11% complete
12% complete
13% complete
14% complete
15% complete
16% complete
17% complete
18% complete
19% complete
20% complete
21% complete
22% complete
23% complete
24% complete
25% complete
26% complete
27% complete
28% complete
29% complete
30% complete
31% complete
32% complete
33% complete
34% complete
35% complete
36% complete
37% complete
38% complete
39% complete
40% complete
41% complete
42% complete
43% complete
44% complete
45% complete
46% complete
47% complete
48% complete
49% complete
50% complete
51% complete
52% complete
53% complete
54% complete
55% complete
56% complete
57% complete
58% complete
59% complete
60% complete
61% complete
62% complete
63% complete
64% complete
65% complete
66% complete
67% complete
68% complete
69% complete
70% complete
71% complete
72% complete
73% complete
74% complete
75% complete
76% complete
77% complete
78% comp

Vamos gravar e dar uma vista em alguns resultados.

In [4]:
recipients.to_csv('recipients.csv')
recipients

,count,send day,send time
id,,,
fc7edfc56268400016851d22245ff57c66956b25,930,3,17:30
c5b8c2183afec62b5384ea6c02f14f769bc4a61f,365,3,12:00
2cd0335e35aa7b216a8cb15aaf5d844041d5c3d2,264,5,10:30
7a1ffa6b95b204188416133f2987eb1525f62211,181,3,12:00
d6fd732b813bbadc8c1c21e7e2374c6d3cdc52df,177,3,17:30
3157698d47d1b17609465f853f70e29cf5ec4795,176,3,17:30
e42bee202af8ef72213e3b3087f482e1ab685bfc,176,4,18:15
62a31a7468e3079e331cbb8886cd630648d888bf,154,3,10:00
addfc47ac6591044f01a02384aeebf4c00677729,147,5,10:30
